In [ ]:
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, StackingClassifier

from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
X_test = np.load("X_test.npy")
y_test = pd.read_csv("y_test.csv").values.ravel()
X_train = np.load("X_train.npy")
y_train = pd.read_csv("y_train.csv").values.ravel()

lr = joblib.load("logistic_regression_model.joblib")
rf = joblib.load("random_forest_model.joblib")
xgb = joblib.load("xgboost_model.joblib")
nb = joblib.load("naive_bayes_model.joblib")

In [ ]:
voting_model = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('xgb', xgb),
        ('nb', nb)
    ],
    voting='soft'
)

voting_model.fit(X_train, y_train)

y_pred_voting = voting_model.predict(X_test)
y_prob_voting = voting_model.predict_proba(X_test)[:, 1]

print("Ensemble (Soft Voting) - Test Evaluation:")
print(classification_report(y_test, y_pred_voting))
print("AUC-ROC:", roc_auc_score(y_test, y_prob_voting))

In [ ]:
stack_model = StackingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('xgb', xgb),
        ('nb', nb)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

stack_model.fit(X_train, y_train)

In [ ]:
y_pred_stack = stack_model.predict(X_test)
y_prob_stack = stack_model.predict_proba(X_test)[:, 1]

print("Stacking Classifier - Test Evaluation:")
print(classification_report(y_test, y_pred_stack))
print("AUC-ROC:", roc_auc_score(y_test, y_prob_stack))

In [ ]:
joblib.dump(stack_model, "stacking_model_final.joblib")
print("Stacking model saved as 'stacking_model_final.joblib'")